# Question 1

## a)

In [1]:
from composite import Composite, CompositeType
import numpy as np

print("Matrice de rigidité 3x3 (GPA)=")
print(CompositeType.Glass_Epoxy.q_3x3_matrix / 10 ** 9)
print("Matrice de souplesse 3x3 (TPa^-1)=")
print(CompositeType.Glass_Epoxy.s_3x3_matrix / 10 ** -12)

Matrice de rigidité 3x3 (GPA)=
[[51.00026119  3.93803617  0.        ]
 [ 3.93803617 15.5040794   0.        ]
 [ 0.          0.          4.7       ]]
Matrice de souplesse 3x3 (TPa^-1)=
[[ 20.          -5.08         0.        ]
 [ -5.08        65.78947368   0.        ]
 [  0.           0.         212.76595745]]


## b)

In [2]:
composite = Composite(angle=0, composite_type=CompositeType.Glass_Epoxy, delta_t=75)

print("Déformations=", composite.solve_strains_and_stresses(strains=(None, None, None), stresses=(20, 10, -5)))
print("epsilon_3=", composite.epsilon_3(stresses=(20, 10, -5)))

Déformations= {epsilon_x: 824.700000000000, epsilon_y: 2303.79473684211, gamma_xy: -1063.82978723404}
epsilon_3= -0.000383178947368421


# Question 2

## a)

In [3]:
radial_composite = Composite(angle=53, composite_type=CompositeType.Graphite_Epoxy, delta_t=-150, delta_m=1)
radial_stresses = radial_composite.solve_radial_stresses(pressure=1.2, diameter=0.5, thickness=8)
print(f"Contraintes radiales: {radial_stresses}")

Contraintes radiales: {'sigma_a': 18.75, 'sigma_h': 37.5}


## b)

In [4]:
print(f"Coefficients hydriques globaux: {np.array(radial_composite.global_hygroscopic_coeffs) / 10 ** -6}")
print(f"Coefficients thermiques globaux:{np.array(radial_composite.global_thermal_coeffs) / 10 ** -6}")

Coefficients hydriques globaux: [ 3095.27356665  1820.72643335 -4444.87408202  4770.        ]
Coefficients thermiques globaux:[ 15.49247461   8.78952539 -23.37596192  24.3       ]


## c)

In [5]:
print("Matrice Sbar (TPa^-1)=")
print(radial_composite.global_s_matrix / 10 ** -12)
print("Matrice Qbar (GPA)=")
print(radial_composite.global_q_matrix / 10 ** 9)

Matrice Sbar (TPa^-1)=
[[ 86.22932093 -32.78037981 -18.73905343]
 [-32.78037981  65.22767969 -54.50237358]
 [-18.73905343 -54.50237358 102.55120803]]
Matrice Qbar (GPA)=
[[30.83524587 36.34381052 24.94994751]
 [36.34381052 70.41385199 44.06355381]
 [24.94994751 44.06355381 37.72853333]]


## d)

In [6]:
from sympy import symbols
epsilon_x, epsilon_y, gamma_xy = symbols('epsilon_x epsilon_y gamma_xy')

global_strains = radial_composite.solve_strains_and_stresses(strains=(None, None, None), stresses=(radial_stresses['sigma_a'], radial_stresses['sigma_h'], 0))
global_mechanical_strains = radial_composite.mechanical_strains(values=(global_strains[epsilon_x], global_strains[epsilon_y], global_strains[gamma_xy]))
global_mechanical_strains = global_mechanical_strains / 10 ** -6
print("Déformations globales:")
print(global_strains)
print("Déformations mécaniques globales:")
print(global_mechanical_strains / 10 ** -6)

Déformations globales:
{epsilon_x: 1158.93789983545, epsilon_y: 2333.70349172889, gamma_xy: -3333.67605465999}
Déformations mécaniques globales:
[387535524.593382 1831405866.97096 -2395196260.91541]


## e)

In [7]:
print("Déformations locales:")
print(radial_composite.global_to_local_strains((global_strains[epsilon_x], global_strains[epsilon_y], global_strains[gamma_xy])))
print("Déformations mécaniques locales:")
print(radial_composite.global_to_local_strains((global_mechanical_strains[0], global_mechanical_strains[1], global_mechanical_strains[2])))

Déformations locales:
[305.957787503607 3186.68360406073 2048.14281805040]
Déformations mécaniques locales:
[157.257787503607 2061.68360406073 2048.14281805040]


## f)

In [8]:
print("epsilon_z:")
print(radial_composite.epsilon_z(stresses=(radial_stresses['sigma_a'], radial_stresses['sigma_h'], 0)) / 10 ** -6)

epsilon_z:
109.11072036846707


## g)

In [9]:
print("Contraintes locales (MPa):")
sigma_1, sigma_2, tau_12 = radial_composite.global_to_local_stresses((radial_stresses['sigma_a'], radial_stresses['sigma_h'], 0))
print(sigma_1, sigma_2, tau_12)

Contraintes locales (MPa):
30.709100210784367 25.54089978921563 9.011828399421738


## h)

In [10]:
fs_max = radial_composite.fs_max((sigma_1, sigma_2, tau_12))
print("Facteur de sécurité maximal:", fs_max)
fs_tsai_hill = radial_composite.fs_tsai_hill((sigma_1, sigma_2, tau_12))
print("Facteur de sécurité Tsai-Hill:", fs_tsai_hill)
fs_tsai_wu = radial_composite.fs_tsai_wu((sigma_1, sigma_2, tau_12))
print("Facteur de sécurité Tsai-Wu:", fs_tsai_wu)

Facteur de sécurité maximal: 1.957644421795663
Facteur de sécurité Tsai-Hill: [-1.92762021151398, 1.92762021151398]
Facteur de sécurité Tsai-Wu: [-7.51603697075040, 1.95286643554009]
